In [15]:
import csv
import sqlite3
# import functions
import pandas as pd


In [16]:
df = pd.read_csv('Resources/airlines_reviews.csv')
df.head(2)

,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended
0,Flight was amazing,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Flight was amazing. The crew onboard this fl...,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes
1,seats on this aircraft are dreadful,Robert Watson,2024-02-21,Singapore Airlines,True,Booking an emergency exit seat still meant h...,Solo Leisure,February 2024,Phuket to Singapore,Economy Class,5,3,4,4,1,3,no


In [17]:
def split_to_via(df, column):
    # Split the text based on separators (to, via)
    split_values = df[column].str.split(r'(?<=\bto\b)|(?<=\bvia\b)', expand=True)
    
    # remove to and via from the split columns
    split_values = split_values.apply(lambda x: x.str.replace(r'\bto\b|\bvia\b', '', regex=True))
    
    # Create new columns and load in the split values
    df['Origin'] = split_values[0].str.strip()
    df['Destination'] = split_values[1].str.split().str.join(' ').str.strip()
    df['Via'] = split_values[2].str.split().str.join(' ').str.strip()
    
    return df

split_to_via(df, 'Route')

,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,Origin,Destination,Via
0,Flight was amazing,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Flight was amazing. The crew onboard this fl...,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes,Jakarta,Singapore,None
1,seats on this aircraft are dreadful,Robert Watson,2024-02-21,Singapore Airlines,True,Booking an emergency exit seat still meant h...,Solo Leisure,February 2024,Phuket to Singapore,Economy Class,5,3,4,4,1,3,no,Phuket,Singapore,None
2,Food was plentiful and tasty,S Han,2024-02-20,Singapore Airlines,True,Excellent performance on all fronts. I would...,Family Leisure,February 2024,Siem Reap to Singapore,Economy Class,1,5,2,1,5,10,yes,Siem Reap,Singapore,None
3,“how much food was available,D Laynes,2024-02-19,Singapore Airlines,True,Pretty comfortable flight considering I was f...,Solo Leisure,February 2024,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,yes,Singapore,London Heathrow,None
4,“service was consistently good”,A Othman,2024-02-19,Singapore Airlines,True,The service was consistently good from start ...,Family Leisure,February 2024,Singapore to Phnom Penh,Economy Class,5,5,5,5,5,10,yes,Singapore,Phnom Penh,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,an uneventful flight,N Vickers,2016-06-20,Korean Air,True,"KE124, Brisbane to Incheon (A330) and KE867,...",Business,June 2016,BNE to ULN via ICN,Economy Class,5,4,5,3,4,7,yes,BNE,ULN,ICN
8096,Korean Air always impresses,Kim Holloway,2016-06-12,Korean Air,False,Our recent flight was our fourth trip to the...,Couple Leisure,June 2016,SYD to LHR via ICN,Economy Class,3,5,5,4,5,10,yes,SYD,LHR,ICN
8097,didn’t offer anything,C Clark,2016-06-06,Korean Air,True,I flew Korean Air from Bali to Seoul in Pres...,Business,April 2016,DPS to ICN,Business Class,4,5,5,5,1,2,no,DPS,ICN,None
8098,appreciated the service onboard,E Petan,2016-04-21,Korean Air,False,Seoul to Paris with Korean Air. I am traveli...,Business,April 2016,ICN to CDG,Business Class,5,1,3,4,5,10,yes,ICN,CDG,None


In [18]:
df.tail()

,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,Origin,Destination,Via
8095,an uneventful flight,N Vickers,2016-06-20,Korean Air,True,"KE124, Brisbane to Incheon (A330) and KE867,...",Business,June 2016,BNE to ULN via ICN,Economy Class,5,4,5,3,4,7,yes,BNE,ULN,ICN
8096,Korean Air always impresses,Kim Holloway,2016-06-12,Korean Air,False,Our recent flight was our fourth trip to the...,Couple Leisure,June 2016,SYD to LHR via ICN,Economy Class,3,5,5,4,5,10,yes,SYD,LHR,ICN
8097,didn’t offer anything,C Clark,2016-06-06,Korean Air,True,I flew Korean Air from Bali to Seoul in Pres...,Business,April 2016,DPS to ICN,Business Class,4,5,5,5,1,2,no,DPS,ICN,None
8098,appreciated the service onboard,E Petan,2016-04-21,Korean Air,False,Seoul to Paris with Korean Air. I am traveli...,Business,April 2016,ICN to CDG,Business Class,5,1,3,4,5,10,yes,ICN,CDG,None
8099,genuinely friendly staff,D Lanor,2016-04-12,Korean Air,False,The 13 hour flight in Business class from Se...,Business,April 2016,ICN to YYZ,Business Class,3,5,3,3,5,10,yes,ICN,YYZ,None


In [19]:
print(f'Rows and Columns: {df.shape}')
df.columns

Rows and Columns: (8100, 20)


Index(['Title', 'Name', 'Review Date', 'Airline', 'Verified', 'Reviews',
       'Type of Traveller', 'Month Flown', 'Route', 'Class', 'Seat Comfort',
       'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
       'Value For Money', 'Overall Rating', 'Recommended', 'Origin',
       'Destination', 'Via'],
      dtype='object')

In [20]:
airport_codes = pd.read_csv('Resources/airports_utf.csv')

In [29]:
df_singapore = df[df['Origin'].str.contains('Singapore')]

# Display the filtered DataFrame
df_singapore

,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,Origin,Destination,Via
3,“how much food was available,D Laynes,2024-02-19,Singapore Airlines,True,Pretty comfortable flight considering I was f...,Solo Leisure,February 2024,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,yes,Singapore,London Heathrow,None
4,“service was consistently good”,A Othman,2024-02-19,Singapore Airlines,True,The service was consistently good from start ...,Family Leisure,February 2024,Singapore to Phnom Penh,Economy Class,5,5,5,5,5,10,yes,Singapore,Phnom Penh,None
5,“seat is absolutely dreadful”,Robert Watson,2024-02-19,Singapore Airlines,True,This flight was over six hours long on a B737...,Solo Leisure,February 2024,Singapore to Cairns,Economy Class,5,1,5,5,1,3,no,Singapore,Cairns,None
6,Very ordinary service,S Holger,2024-02-19,Singapore Airlines,True,"Boarding process went smoothly, and plane le...",Solo Leisure,February 2024,Singapore to Frankfurt,Economy Class,1,3,1,2,2,5,no,Singapore,Frankfurt,None
7,I like flying this airline,A Jabil,2024-02-19,Singapore Airlines,True,Pleasant flight which operated on time. Chec...,Family Leisure,February 2024,Singapore to Bandar Seri Begawan,Economy Class,2,5,3,4,5,10,yes,Singapore,Bandar Seri Begawan,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7931,crew were efficient and friendly,Kah Kay Au,2023-04-09,Korean Air,True,I was in Prestige Class on the Boeing 777. K...,Solo Leisure,April 2023,Singapore to Incheon,Business Class,4,4,5,2,4,7,yes,Singapore,Incheon,None
7942,impressed with the spacious cabin,T Barden,2022-08-08,Korean Air,True,Smooth efficient (and friendly) check-in. Wa...,Business,May 2022,Singapore to Seoul,Economy Class,5,5,1,2,5,8,yes,Singapore,Seoul,None
8017,would fly Korean Air again,Mohammed Al-Adsany,2018-05-14,Korean Air,True,Business class from Singapore to Seoul. We c...,Solo Leisure,May 2018,Singapore to Seoul,Business Class,5,5,1,1,4,8,yes,Singapore,Seoul,None
8020,the food was below par,S Maravasan,2018-04-23,Korean Air,True,Singapore to Seoul. Flight arrived either ea...,Business,April 2018,Singapore to Seoul,Economy Class,3,1,5,5,4,6,yes,Singapore,Seoul,None


In [25]:
singapore_routes = df['Route'][df['Route'].str.contains('Singapore')]

# Display the extracted values
singapore_routes

0                  Jakarta to Singapore
1                   Phuket to Singapore
2                Siem Reap to Singapore
3          Singapore to London Heathrow
4               Singapore to Phnom Penh
                     ...               
8017                 Singapore to Seoul
8020                 Singapore to Seoul
8044               Incheon to Singapore
8061                 Seoul to Singapore
8071    Singapore to Honolulu via Seoul
Name: Route, Length: 1069, dtype: object

In [28]:
unique_singapore_origin = df['Origin'][df['Origin'].str.contains('Singapore')].unique()

# Display the unique values
unique_singapore_origin

array(['Singapore', 'Singapore Changi'], dtype=object)

In [21]:
def lookup_code(airport_string):
    if len(airport_string) == 3 and airport_string.isupper():
        return airport_string
    else: 
        code = airport_codes.loc[airport_codes["City"] == airport_string, "Code"]
        if code.empty:    # because .loc returns series (vs string), can't run "not code" instead run code.empty
            print(f'could not find code for {airport_string}')
            return None
        display(code)
        return code.iloc[0]
    
   

In [23]:
df.Origin.apply(lookup_code)

could not find code for Jakarta


1308    HKT
Name: Code, dtype: object

1553    SAI
Name: Code, dtype: object

could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore


1553    SAI
Name: Code, dtype: object

...

1278    PAR
Name: Code, dtype: object

1734    ULN
Name: Code, dtype: object

could not find code for Seoul
could not find code for Seoul
could not find code for Singapore


108    AKL
Name: Code, dtype: object

0       None
1        HKT
2        SAI
3       None
4       None
        ... 
8095     BNE
8096     SYD
8097     DPS
8098     ICN
8099     ICN
Name: Origin, Length: 8100, dtype: object

In [ ]:
df['Origin'].value_counts()

In [ ]:
def lookup_code(airport_string):
    if len(airport_string) == 3 and airport_string.isupper():
        return airport_string
    else: 
        code = airport_codes.loc[airport_codes["City"] == airport_string, "Code"]
        if code.empty:    # because .loc returns series (vs string), can't run "not code" instead run code.empty
            print(f'could not find code for {airport_string}')
            return None
        display(code)
        return code.iloc[0]

In [ ]:
df['O'].value_counts()

In [ ]:
lookup_code("Fukuoka")

In [ ]:
df['Origin'].apply(functions.lookup_code)

In [ ]:
airport_codes.head()

In [ ]:
def lookup_city(airport_string):
    if len(airport_string) == 3 and not airport_string.isupper():
        # city = airport_codes.loc[airport_codes['Code'] == airport_string, 'City']
        return airport_string
    else: 
        city = airport_codes.loc[airport_codes["Code"] == airport_string, "City"]
        if city.empty:  
            print(f'could not find city for {airport_string}')
            return None
        display(city)
        return city.iloc[0]
        

In [ ]:
lookup_city("JFK")

In [ ]:
### create new columns for "origin_code" "origin_city" "destination_code" "destination_city" "via_code" "via_city"
df[:2]

In [13]:
df.insert(18,'origin_code',df['Origin'].apply(functions.lookup_code))

could not find code for Jakarta


1754    HKT
Name: Code, dtype: object

could not find code for Siem Reap
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Singapore
could not find code for Siem Reap


1714    BWN
Name: Code, dtype: object

could not find code for Singapore
could not find code for London Heathrow
could not find code for Singapore
could not find code for Singapore
could not find code for Perth
could not find code for Sydney


...

667    ROM
Name: Code, dtype: object

could not find code for Istanbul
could not find code for Istanbul


491    WAW
Name: Code, dtype: object

could not find code for Singapore
could not find code for Los Angeles
could not find code for London Gatwick
could not find code for Istanbul
could not find code for Kuala Lumpur


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [15]:
df.origin_code

0       None
1        HKT
2       None
3       None
4       None
        ... 
8095     BNE
8096     SYD
8097     DPS
8098     ICN
8099     ICN
Name: origin_code, Length: 8100, dtype: object

In [ ]:
for i,city in enumerate(df.Origin.value_counts().index):
    if len(city) == 3:
        print(city)

In [ ]:
df.Origin

In [ ]:
### lookup failures, only return cities not found, if return series call unique.apply - return nothing if you can't find it

### get lats/long for origin-destination-via

### https://www.developer.aero/api-catalog/flight-duration-overview

In [ ]:
# airports2 = airports.replace(', ','_').replace(' ',',').replace('\t',',')

In [ ]:
# import csv

In [ ]:
# airports2

In [ ]:
airports = [ airport for airport in df.Origin.value_counts().index if len(airport) == 3]
len(airports)

In [ ]:
df_updated = df[['Title', 'Airline', 'Reviews',
       'Type of Traveller', 'Month Flown', 'Route', 'Class', 'Seat Comfort',
       'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
       'Value For Money', 'Overall Rating', 'Recommended']]

In [ ]:
ratings = list(df.iloc[:,16].values)

In [ ]:
df.iloc[17]

In [ ]:
with open('Resources/airlines_reviews.csv', 'r') as file:
    reader = csv.reader(file)
    # Skip the header row if it exists
    next(reader, None)

    # Iterate over the rows and extract the comments
    comments = [row[0] for row in reader]

    ratings = [float(row[16]) for row in reader]

polarities = []
subjectivities = []

# Analyze the sentiment of each comment
for comment in comments:
    blob = TextBlob(comment)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    polarities.append(polarity)
    subjectivities.append(subjectivity)
    
    print(f"Comment: {comment}")
    print(f"Polarity: {polarity}")
    print(f"Subjectivity: {subjectivity}")
    print()

In [ ]:
ratings

In [ ]:
plt.figure(figsize=(8, 6))
scatter = plt.scatter(polarities, subjectivities, c=ratings, cmap='viridis', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.title('Polarity vs. Subjectivity')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
scatter = plt.scatter(polarities, subjectivities, c=ratings, cmap='viridis', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.title('Polarity vs. Subjectivity')
plt.show()

In [ ]:
df['Name'].nunique()

In [ ]:
df['Title']

In [ ]:
df['Title'].value_counts()

In [ ]:
df['Route'].value_counts()

In [ ]:
df['Airline'].value_counts()

In [ ]:
def add_location_info(df):
    geolocator = Nominatim(user_agent="my_geocoder")

    # Create new columns for city, latitude, and longitude
    df['Origin_City'] = ""
    df['Origin_Latitude'] = 0.0
    df['Origin_Longitude'] = 0.0

    for index, row in df.iterrows():
        location = geolocator.geocode(row['Origin'])
        if location:
            df.at[index, 'Origin_City'] = location.raw.get('display_name').split(",")[-4].strip()
            df.at[index, 'Origin_Latitude'] = location.latitude
            df.at[index, 'Origin_Longitude'] = location.longitude

    return df

df_with_location_info = add_location_info(df)

In [ ]:
# keep routes & split into Origin & Destination, find coord for the locations
# Target "Recommended"
# maybe user can provide the following and 
# Type of Traveller', 'Route', 'Class', 'Seat Comfort',
# 'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
# 'Value For Money